# Animal Identification Resnet18

In [1]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models\
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
data_dir = './Data/animals'
image_datasets = {x: datasets.ImageFolder(os.path.join(
    data_dir, x), data_transforms[x]) for x in ['Train', 'Test']}
dataloaders = {x: torch.utils.data.DataLoader(
    image_datasets[x], batch_size=4, shuffle=True) for x in ['Train', 'Test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Test']}
class_names = image_datasets['Train'].classes

In [8]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

c:\Users\dobao\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\dobao\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
num_epochs = 25
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['Train', 'Test']:
        if phase == 'Train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'Train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'Train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    print()

Epoch 0/24
----------
Train Loss: 1.5048 Acc: 0.4762
Test Loss: 0.3146 Acc: 0.8690

Epoch 1/24
----------
Train Loss: 0.7774 Acc: 0.7245
Test Loss: 0.3931 Acc: 0.8690

Epoch 2/24
----------
Train Loss: 0.6103 Acc: 0.8129
Test Loss: 0.3117 Acc: 0.8929

Epoch 3/24
----------
Train Loss: 0.6801 Acc: 0.7449
Test Loss: 0.2566 Acc: 0.9286

Epoch 4/24
----------
Train Loss: 0.5746 Acc: 0.8061
Test Loss: 0.2858 Acc: 0.8929

Epoch 5/24
----------
Train Loss: 0.7014 Acc: 0.7687
Test Loss: 0.2527 Acc: 0.8929

Epoch 6/24
----------
Train Loss: 0.5438 Acc: 0.8197
Test Loss: 0.3103 Acc: 0.9048

Epoch 7/24
----------
Train Loss: 0.5724 Acc: 0.8129
Test Loss: 0.5462 Acc: 0.8690

Epoch 8/24
----------
Train Loss: 0.7428 Acc: 0.7585
Test Loss: 0.5972 Acc: 0.8810

Epoch 9/24
----------
Train Loss: 0.5930 Acc: 0.7993
Test Loss: 0.3600 Acc: 0.9048

Epoch 10/24
----------
Train Loss: 0.6153 Acc: 0.8299
Test Loss: 0.3538 Acc: 0.9167

Epoch 11/24
----------
Train Loss: 0.5030 Acc: 0.8401
Test Loss: 0.3686 Acc

In [ ]:
torch.save(model.state_dict(), './animal_classifier.pth')